In [1]:
# Import Libriaries
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.ticker as ticker
from sklearn import metrics as m

In [2]:
# Read the dataset
dp = pd.read_csv('House_Price_Prediction.csv')

In [3]:
# Print the dataset
print(dp)

     Unnamed: 0   price   lotsize  bedrooms  bathrms  stories  garagepl  \
0             0   42000  8.674197       3.0        1        2         1   
1             1   38500  8.294050       2.0        1        1         0   
2             2   49500  8.026170       3.0        1        1         0   
3             3   60500  8.802372       3.0        1        2         0   
4             4   61000  8.757784       3.0        1        1         0   
..          ...     ...       ...       ...      ...      ...       ...   
546         546  107500  8.699515       3.0        2        4         1   
547         547  108000  8.699515       3.0        2        3         0   
548         548  113750  8.699515       3.0        1        4         2   
549         549  120000  8.853665       3.0        1        4         2   
550         550   70000  9.464983       3.0        1        1         2   

     driveway_yes  recroom_yes  fullbase_yes  gashw_yes  airco_yes  \
0               1            

In [4]:
# Describe the various columns of the dataset
dp.describe()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
count,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000
mean,275.000000,68445.811252,8.470413,2.967332,1.286751,1.820327,0.698730,0.860254,0.177858,0.346642,0.045372,0.321234,0.232305
std,159.204271,26848.486040,0.399086,0.732880,0.502165,0.881334,0.863386,0.347038,0.382741,0.476333,0.208308,0.467375,0.422686
min,0.000000,25000.000000,7.408531,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,137.500000,49500.000000,8.188689,3.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,275.000000,62500.000000,8.433812,3.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,412.500000,82950.000000,8.757784,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,550.000000,190000.000000,9.692767,6.000000,4.000000,4.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# Check for null values
dp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    551 non-null    int64  
 1   price         551 non-null    int64  
 2   lotsize       551 non-null    float64
 3   bedrooms      551 non-null    float64
 4   bathrms       551 non-null    int64  
 5   stories       551 non-null    int64  
 6   garagepl      551 non-null    int64  
 7   driveway_yes  551 non-null    int64  
 8   recroom_yes   551 non-null    int64  
 9   fullbase_yes  551 non-null    int64  
 10  gashw_yes     551 non-null    int64  
 11  airco_yes     551 non-null    int64  
 12  prefarea_yes  551 non-null    int64  
dtypes: float64(2), int64(11)
memory usage: 56.1 KB


In [6]:
# Check the corrolation between the columns
dp.corr()

,Unnamed: 0,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
Unnamed: 0,1.000000,0.387924,0.386297,0.109361,0.109914,0.248420,0.135595,0.315854,0.095841,-0.013450,-0.036458,0.169776,0.503575
price,0.387924,1.000000,0.560017,0.363247,0.513014,0.435332,0.385734,0.298859,0.253611,0.175110,0.089257,0.462458,0.318696
lotsize,0.386297,0.560017,1.000000,0.151814,0.198791,0.112181,0.365816,0.332750,0.176168,0.035777,-0.015737,0.262216,0.212355
bedrooms,0.109361,0.363247,0.151814,1.000000,0.371325,0.399058,0.136709,-0.010833,0.079088,0.094997,0.045456,0.158087,0.077366
bathrms,0.109914,0.513014,0.198791,0.371325,1.000000,0.322034,0.170263,0.042566,0.122017,0.100567,0.066592,0.187823,0.062495
stories,0.248420,0.435332,0.112181,0.399058,0.322034,1.000000,0.052983,0.125817,0.046397,-0.180526,0.014774,0.312520,0.034156
garagepl,0.135595,0.385734,0.365816,0.136709,0.170263,0.052983,1.000000,0.205116,0.041400,0.046609,0.066032,0.159165,0.087499
driveway_yes,0.315854,0.298859,0.332750,-0.010833,0.042566,0.125817,0.205116,1.000000,0.091646,0.040602,-0.012735,0.109127,0.196923
recroom_yes,0.095841,0.253611,0.176168,0.079088,0.122017,0.046397,0.041400,0.091646,1.000000,0.369287,-0.010181,0.137408,0.159972
fullbase_yes,-0.013450,0.175110,0.035777,0.094997,0.100567,-0.180526,0.046609,0.040602,0.369287,1.000000,0.006119,0.037930,0.231448


In [7]:
# Assigning your independent value
x =dp[["bedrooms"]]

In [8]:
# Assigning your dependent valuey 
y=dp.price

In [9]:
# Splitting your data to training and testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [10]:
# Importing your model
from sklearn.linear_model import LinearRegression
Lr = LinearRegression()
Lr.fit(x_train,y_train) # Training the algorithm

LinearRegression()

In [11]:
# Assigning your predicted test values
y_pred=Lr.predict(x_test)

In [12]:
# Print your predicted test values
print(y_pred)

[ 69547.50154943  84264.19662225  69547.50154943  69547.50154943
  84264.19662225  54830.8064766   69547.50154943  54830.8064766
  69547.50154943  69547.50154943  69547.50154943  69547.50154943
  69547.50154943  69547.50154943  69547.50154943  54830.8064766
  54830.8064766   69547.50154943  69547.50154943  54830.8064766
  69547.50154943  69547.50154943  69547.50154943  69547.50154943
  69547.50154943  54830.8064766   69547.50154943  98980.89169507
  84264.19662225  69547.50154943  69547.50154943  69547.50154943
  84264.19662225  69547.50154943  69547.50154943  69547.50154943
  69547.50154943  54830.8064766   69547.50154943  69547.50154943
  84264.19662225  54830.8064766   69547.50154943  84264.19662225
  69547.50154943  69547.50154943  69547.50154943  84264.19662225
  69547.50154943  69547.50154943  54830.8064766   69547.50154943
  84264.19662225  54830.8064766   69547.50154943  69547.50154943
 113697.5867679   69547.50154943  84264.19662225  84264.19662225
  69547.50154943  84264.1966

In [13]:
m.mean_absolute_error(y_test,y_pred)

17458.24042775344

In [14]:
m.mean_squared_error(y_test,y_pred)

474229174.75232846

In [15]:
np.sqrt(m.mean_squared_error(y_test,y_pred))

21776.803593556342

In [16]:
m.r2_score(y_test,y_pred)  

0.009192877069788996

In [17]:
y_train_pred=Lr.predict(x_train)

In [18]:
m.mean_absolute_error(y_train,y_train_pred)

19486.654946192633

In [19]:
m.mean_squared_error(y_train,y_train_pred)

664340892.0955292

In [20]:
np.sqrt(m.mean_squared_error(y_train,y_train_pred))

25774.81119417811

In [21]:
m.r2_score(y_train,y_train_pred)

0.14813101394231032

In [22]:
x =dp[["bedrooms","lotsize","bathrms"]]

In [23]:
y= dp.price

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [25]:
from sklearn.linear_model import LinearRegression
Lr = LinearRegression()
Lr.fit(x_train,y_train) # Training the algorithm

LinearRegression()

In [26]:
y_pred=Lr.predict(x_test)

In [27]:
m.mean_absolute_error(y_test,y_pred)

13103.564838785556

In [28]:
m.mean_squared_error(y_test,y_pred)

248135271.41011927

In [29]:
np.sqrt(m.mean_squared_error(y_test,y_pred))

15752.31003409085

In [30]:
m.r2_score(y_test,y_pred)  

0.4815709208701311

In [31]:
y_train_pred=Lr.predict(x_train)

In [32]:
m.mean_absolute_error(y_train,y_train_pred)

14713.97928444102

In [33]:
m.mean_squared_error(y_train,y_train_pred)

386073616.4043991

In [34]:
np.sqrt(m.mean_squared_error(y_train,y_train_pred))

19648.756103234606

In [35]:
m.r2_score(y_train,y_train_pred)

0.5049467162669419

In [36]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

In [37]:
rr = Ridge(alpha=0.01)
rr.fit(x_train,y_train)

Ridge(alpha=0.01)

In [38]:
pred_train_rr = rr.predict(x_train)

In [39]:
print('MSE: TRAINING SET')
print(np.sqrt(m.mean_squared_error(y_train,pred_train_rr)))
print('SCORE: TRAINNG SET')
print(r2_score(y_train,pred_train_rr))

MSE: TRAINING SET
19648.756199470165
SCORE: TRAINNG SET
0.5049467114176038


In [40]:
pred_test_rr = rr.predict(x_test)

In [41]:
print('MSE: TESTING SET')
print(np.sqrt(m.mean_squared_error(y_test,pred_test_rr)))
print('SCORE: TESTING SET')
print(r2_score(y_test,pred_test_rr))

MSE: TESTING SET
15752.023872695667
SCORE: TESTING SET
0.4815897565889229


In [42]:
ll = Lasso(alpha=0.01)
ll.fit(x_train,y_train)

Lasso(alpha=0.01)

In [43]:
pred_train_ll = ll.predict(x_train)

In [44]:
print('MSE: TRAINING SET')
print(np.sqrt(m.mean_squared_error(y_train,pred_train_ll)))
print('SCORE: TRAINNG SET')
print(r2_score(y_train,pred_train_ll))

MSE: TRAINING SET
19648.756103256692
SCORE: TRAINNG SET
0.504946716265829


In [45]:
pred_test_ll = ll.predict(x_test)

In [46]:
print('MSE: TESTING SET')
print(np.sqrt(m.mean_squared_error(y_test,pred_test_ll)))
print('SCORE: TESTING SET')
print(r2_score(y_test,pred_test_ll))

MSE: TESTING SET
15752.305353045296
SCORE: TESTING SET
0.4815712289887206


In [47]:
from sklearn.ensemble import RandomForestClassifier
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy",random_state=1)
classifier.fit(x_train, y_train) 


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=1)

In [48]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':range(30),
 'criterion':['gini','entropy']}
c1 = RandomForestClassifier()
grid = GridSearchCV(c1, parameters, cv=10)
grid.fit(x,y)
print(grid.best_params_)
print(grid.best_estimator_)

C:\Users\shice\anaconda\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\shice\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\shice\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\shice\anaconda\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_par

{'criterion': 'gini', 'n_estimators': 21}
RandomForestClassifier(n_estimators=21)


In [49]:
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.02727272727272727
{'criterion': 'gini', 'n_estimators': 21}
RandomForestClassifier(n_estimators=21)
